In [1]:
import pandas as pd
import numpy as np
import string
import json

In [2]:
## Static Config
foreign_key = "Code"

In [3]:
df = pd.read_csv('nep_wp_reservoir_data.csv')

In [4]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
### Fill Empty Integer Values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df.select_dtypes(include=numerics)
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num
df['Yield'] = 0

In [6]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
df[list(df_str)] = df_str

In [7]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [8]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

foreign_key_config = "";
for key in configs:
    if(configs[key].lower() == foreign_key.lower()):
        foreign_key_config = key

configs['foreign_key'] = foreign_key_config

In [9]:
### Replace Dataset Columns
df.columns = index

In [10]:
configs

{'A': 'Code',
 'B': 'WSS Name',
 'C': 'Reservoir RVT No in the Scheme',
 'D': 'Reservoir RVT Construction',
 'E': 'Reservoir RVT Type',
 'F': 'Reservoir Lat',
 'G': 'Reservoir Lon',
 'H': 'Reservoir Elv',
 'I': 'Reservori RVT Capacity (cum)',
 'J': 'Reservoir Adequacy of RVT',
 'K': 'Reservoir RVT Condition',
 'L': 'Reservoir Earthquake Effect',
 'M': 'Yield',
 'foreign_key': 'A'}

In [11]:
data = list(df.T.to_dict().values())
data[:2]

[{'A': 61005025,
  'B': 'Hathisude WSS',
  'C': 'RVT1',
  'D': 0,
  'E': 'RCC',
  'F': 28,
  'G': 81,
  'H': 600,
  'I': 18,
  'J': 'Adequate',
  'K': 'Good',
  'L': 'None',
  'M': 0},
 {'A': 61005029,
  'B': 'BhulkeKapre Water Supply System',
  'C': 'RVT1',
  'D': 0,
  'E': 'Masonary',
  'F': 28,
  'G': 81,
  'H': 559,
  'I': 9,
  'J': 'Adequate',
  'K': 'Minor Repair',
  'L': 'None',
  'M': 0}]

In [12]:
### to json
df.to_json('../resources/js/data/nep_wp_reservoir_data.json', orient="records", indent=2)

with open('../resources/js/data/nep_wp_reservoir_config.json', 'w') as outfile:
    json.dump(configs, outfile)